In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import cifar10

from defense.load_classifier import load_classifier
from defense.detector import get_train_stats, kl_test

Using TensorFlow backend.


# load CIFAR10 data

In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
Y_train = keras.utils.to_categorical(Y_train, 10)
Y_test = keras.utils.to_categorical(Y_test, 10)
source_samples, img_rows, img_cols, channels = X_test.shape
nb_classes = Y_test.shape[1]

num_test = 1000
batch_size = 200
num_batches = num_test // batch_size
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32)
X_test = X_test[:num_test] / 255.0
Y_test = Y_test[:num_test]

# load model

In [ ]:
np.random.seed(1234)
tf.set_random_seed(1234)

# Create TF session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = keras.backend.get_session()

model_name = "fea_K10_F_mid"
data_name = "cifar10"

# Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(batch_size, img_rows, img_cols, channels))
y = tf.placeholder(tf.float32, shape=(batch_size, nb_classes))

# Define TF model graph  
model = load_classifier(sess, model_name, data_name, path='models/cifar10_conv_vae_fea_F_mid', fea_weights='models/cifar10vgg.h5')
keras.backend.set_learning_phase(0)

# output logits
preds = model.predict(x, softmax=False)

# get training stats for the detector
train_stats = get_train_stats(sess, model, x, X_train, Y_train, batch_size=batch_size)

# random attack 

This attack isn't particularly effective. 
The main point is to show that the classifier's Monte Carlo sampling is brittle.

In [4]:
eps = 8.0/255.0

X = X_test[:batch_size]
Y = np.argmax(Y_test[:batch_size], axis=-1)
all_X_adv = X.copy()

success = np.zeros(batch_size).astype(np.bool)
for i in range(100):
    noise = np.random.uniform(low=-eps, high=eps, size=X.shape)
    X_adv = np.clip(X + noise, 0, 1)
    Y_pred = sess.run(tf.argmax(preds, axis=-1), feed_dict={x: X_adv})
    
    success |= (Y_pred != Y)
    all_X_adv[Y_pred != Y] = X_adv[Y_pred != Y]
    
    if i % 10 == 0:
        print("fraction of inputs with at least one failure in {} trials: {:.1f}%".format(i, 100*np.mean(success)))
        

# check if the examples are still adversarial with different inference-time randomness
Y_pred = sess.run(tf.argmax(preds, axis=-1), feed_dict={x: all_X_adv})
print("single-shot success: {:.1f}%".format(100*np.mean(Y_pred != Y)))

fraction of inputs with at least one failure in 0 trials: 11.0%
fraction of inputs with at least one failure in 10 trials: 22.5%
fraction of inputs with at least one failure in 20 trials: 24.0%
fraction of inputs with at least one failure in 30 trials: 26.5%
fraction of inputs with at least one failure in 40 trials: 28.0%
fraction of inputs with at least one failure in 50 trials: 29.0%
fraction of inputs with at least one failure in 60 trials: 29.0%
fraction of inputs with at least one failure in 70 trials: 30.0%
fraction of inputs with at least one failure in 80 trials: 30.5%
fraction of inputs with at least one failure in 90 trials: 30.5%
single-shot success: 14.5%


# Decompose the different parts of the model and loss function

This will help us understand what's the best way to attack the defense

In [10]:
from defense.lowerbound_functions import lowerbound_F as bound_func
from defense.lowerbound_functions import encoding, log_gaussian_prob
from defense.vgg_cifar10 import cifar10vgg

# first build the feature extractor
cnn = cifar10vgg('models/cifar10vgg.h5', train=False)
N_layer = 36
    
def feature_extractor(x):
    out = cnn.normalize_production(x * 255.0)
    for i in range(N_layer):
        out = cnn.model.layers[i](out)
    if len(out.get_shape().as_list()) == 4:
        out = tf.reshape(out, [x.get_shape().as_list()[0], -1])
    return out

X = X_test[:batch_size]
Y = Y_test[:batch_size]
K = 10

# features extracted from VGG
fea = feature_extractor(x) # B x 512
fea_np = sess.run(fea, feed_dict={x: X})

# get the latent variables
_, enc_mlp = model.enc
z, logq = encoding(enc_mlp, fea, y, K) # z is (B*K) x 128, logq is (B*K)
z_np, logq_np = sess.run([z, logq], feed_dict={x: X, y: Y})

# decode to an input
pyz, pxz = model.dec
mu_x = pxz(z) # (B*K) x 32 x 32 x 3
mu_x_np = sess.run(mu_x, feed_dict={x: X, y: Y})
mu_x_np = mu_x_np.reshape(K, batch_size, -1)

x_rep = tf.tile(fea, [K, 1])
y_rep = tf.tile(y, [K, 1])

# log prior
log_prior_z = log_gaussian_prob(z, 0.0, 0.0)

# reconstruction loss
ind = list(range(1, len(x_rep.get_shape().as_list())))
logp = -tf.reduce_sum((x_rep - mu_x)**2, ind)

# cross-entropy loss
logit_y = pyz(z)
log_pyz = -tf.nn.softmax_cross_entropy_with_logits(labels=y_rep, logits=logit_y) 

# full score
score = logp * 1.0 + log_pyz + (log_prior_z - logq)

# pick a random class
Y_rand = np.zeros((len(X), nb_classes), dtype=np.float32)
Y_rand[np.arange(len(X)), np.random.choice(nb_classes, len(X))] = 1 

# compute all scores for the correct class, and for a random class
results_correct = sess.run([logp, log_pyz, log_prior_z, logq], feed_dict={x: X, y: Y})
results_incorrect = sess.run([logp, log_pyz, log_prior_z, logq], feed_dict={x: X, y: Y_rand})

score_correct = sess.run(score, feed_dict={x: X, y: Y})
score_incorrect = sess.run(score, feed_dict={x: X, y: Y_rand})

print("full score:\t {:.1f}\t{:.1f}".format(np.mean(score_correct[:K]), np.mean(score_incorrect[:K])))

print("logp:\t\t {:.1f}\t{:.1f}".format(np.mean(results_correct[0][:K]), np.mean(results_incorrect[0][:K])))

print("log_pyz:\t {:.1f}\t\t{:.1f}".format(np.mean(results_correct[1][:K]), np.mean(results_incorrect[1][:K])))

print("log_prior_z:\t {:.1f}\t\t{:.1f}".format(np.mean(results_correct[2][:K]), np.mean(results_incorrect[2][:K])))

print("logq:\t\t {:.1f}\t\t{:.1f}".format(np.mean(results_correct[3][:K]), np.mean(results_incorrect[3][:K])))

full score:	 -1535.1	-1609.6
logp:		 -1308.8	-1319.2
log_pyz:	 -0.0		-67.1
log_prior_z:	 -176.6		-176.8
logq:		 46.2		40.3


# Simply try to optimize the full loss

We don't quite break the classifier this way (we get down to 22% accuracy or so)

In [11]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=preds) 
g = tf.gradients(loss, x)[0]

eps = 8.0/255.0
N = 100
step_size = 4 * eps / N

X_test_adv = X_test[:num_batches*batch_size].copy()
Y_test_adv = Y_test.copy()

for j in range(num_batches):
    X = X_test[j*batch_size: (j+1)*batch_size]
    Y = Y_test[j*batch_size: (j+1)*batch_size]
    
    X_adv = X + np.random.uniform(low=-eps, high=eps, size=X.shape)
    X_adv = np.clip(X_adv, 0, 1)

    for i in range(N+1):
        preds_np, loss_np, g_np = sess.run([preds, loss, g], feed_dict={x: X_adv, y: Y})
        if i % 20 == 0:
            print(j, i, np.mean(loss_np), np.mean(np.argmax(preds_np, axis=-1) != np.argmax(Y, axis=-1)))
        X_adv = X_adv + step_size * np.sign(g_np)
        X_adv = np.clip(X_adv, X-eps, X+eps)
        X_adv = np.clip(X_adv, 0, 1)
    
    X_test_adv[j*batch_size: (j+1)*batch_size] = X_adv.copy()
    Y_test_adv[j*batch_size: (j+1)*batch_size] = preds_np

succ = np.mean(np.argmax(Y_test_adv, axis=-1) != np.argmax(Y_test, axis=-1))
print("Attack success on classifier: {:.1f}%".format(100.0 * succ))

0 0 2.479561 0.115
0 20 33.412643 0.52
0 40 68.76878 0.675
0 60 97.9147 0.725
0 80 118.84158 0.75
0 100 132.82681 0.775
1 0 2.3799384 0.115
1 20 36.94336 0.54
1 40 73.00916 0.66
1 60 103.21084 0.715
1 80 125.24668 0.765
1 100 142.29276 0.785
2 0 1.1557727 0.08
2 20 35.242096 0.475
2 40 71.52065 0.635
2 60 103.37664 0.7
2 80 125.70251 0.735
2 100 145.7627 0.76
3 0 2.8797443 0.145
3 20 34.308956 0.49
3 40 70.13879 0.64
3 60 99.82238 0.71
3 80 122.891525 0.765
3 100 142.81285 0.785
4 0 1.4962208 0.095
4 20 34.98964 0.495
4 40 70.03624 0.64
4 60 99.13672 0.695
4 80 123.43363 0.745
4 100 138.7791 0.775
Attack success on classifier: 77.6%


# Attack on the log_pyz term in the loss

This attack achieves around 98% success on the classifier, 
and only 12% of adversarial examples are correctly detected 

In [20]:
mu_qz_correct, _ = enc_mlp(fea, y)

# take more random samples here to get adv examples that work with very high confidence
K = 100
z, _ = encoding(enc_mlp, fea, y, K)
y_rep = tf.tile(y, [K, 1])

# let's start with something simple and just minimize the logit of the true class
logit_y = pyz(z)
true_logit = tf.reduce_sum(logit_y * y_rep, axis=-1)
loss = -true_logit
g = tf.gradients(loss, x)[0]

# this attack actually works better with a small epsilon, 
# because the changes in the classifier's logits are small enough
# that the KL-detector is often fooled. With eps=8/255, this attack
# generates over-confident adversarial examples that are easily detected
eps = 4.0/255.0
N = 200
step_size = 4 * eps / N

X_test_adv = X_test[:num_batches*batch_size].copy()
Y_test_adv = Y_test.copy()

for j in range(num_batches):
    X = X_test[j*batch_size: (j+1)*batch_size]
    Y = Y_test[j*batch_size: (j+1)*batch_size]
    
    X_adv = X + np.random.uniform(low=-eps, high=eps, size=X.shape)
    X_adv = np.clip(X_adv, 0, 1)

    for i in range(N):
        loss_np, g_np, preds_np = sess.run([loss, g, preds], feed_dict={x: X_adv, y: Y})
        if i % 20 == 0:
            print(i, np.mean(loss_np), np.mean(np.argmax(preds_np, axis=-1) != np.argmax(Y, axis=-1)))
        X_adv = X_adv + step_size * np.sign(g_np)
        X_adv = np.clip(X_adv, X-eps, X+eps)
        X_adv = np.clip(X_adv, 0, 1)
    
    X_test_adv[j*batch_size: (j+1)*batch_size] = X_adv.copy()
    Y_test_adv[j*batch_size: (j+1)*batch_size] = preds_np

succ = np.mean(np.argmax(Y_test_adv, axis=-1) != np.argmax(Y_test, axis=-1))
print("Attack success on classifier: {:.1f}%".format(100.0 * succ))

# 
# Check how many of our adversarial examples on the classifier are detected (at a 5% FP rate)
#
y_logits_adv = Y_test_adv
y_adv = np.zeros((y_logits_adv.shape[0], nb_classes), dtype=np.float32)
y_adv[np.arange(y_logits_adv.shape[0]), np.argmax(y_logits_adv, 1)] = 1 
tps = kl_test(y_adv, y_logits_adv, np.ones(len(y_logits_adv), dtype=np.bool), train_stats)
print("TP rate for adv examples: {:.1f}%".format(100 * np.mean(tps)))

0 -59.71122 0.11
20 -19.711107 0.51
40 2.8009655 0.785
60 15.527143 0.9
80 23.398348 0.94
100 28.106821 0.96
120 30.783594 0.95
140 32.641933 0.97
160 33.899258 0.965
180 34.801674 0.97
0 -60.5066 0.075
20 -19.918486 0.495
40 2.037089 0.79
60 14.713792 0.89
80 22.590199 0.925
100 27.267298 0.95
120 30.023588 0.955
140 31.901882 0.955
160 33.163776 0.96
180 34.066242 0.97
0 -65.11866 0.075
20 -22.559828 0.43
40 1.5491282 0.765
60 15.371693 0.93
80 23.71495 0.96
100 28.598688 0.975
120 31.472988 0.985
140 33.422874 0.99
160 34.787827 0.99
180 35.774197 0.985
0 -60.071743 0.11
20 -19.364565 0.5
40 3.2485545 0.815
60 16.613098 0.925
80 24.998487 0.95
100 30.043682 0.965
120 32.981243 0.96
140 35.01301 0.98
160 36.36 0.975
180 37.32801 0.98
0 -60.10262 0.08
20 -19.83017 0.51
40 2.5946622 0.78
60 15.57872 0.905
80 23.454977 0.945
100 28.025974 0.955
120 30.687212 0.97
140 32.477238 0.97
160 33.720547 0.96
180 34.581184 0.97
Attack success on classifier: 97.8%
TP rate for adv examples: 11.8%


# Adaptive attack against the classifier and detector.

In [24]:
# Pick a larger number of samples to get better gradient estimates
K = 100

# Let's compute the logit of the true class
mu_qz_correct, _ = enc_mlp(fea, y)
z, _ = encoding(enc_mlp, fea, y, K)
y_rep = tf.tile(y, [K, 1])
logit_y = pyz(z)
true_logit = tf.reduce_sum(logit_y * y_rep, axis=-1)

# We'll simply try to align the adversarial logits with the clean logits of a different class.
# This hits two birds with one stone! We'll make sure that the adversarial example is misclassified 
# (as the top logit is changed), and that it is undetected (as the logits are distributed as in a clean example)
y_target = tf.placeholder(tf.float32, shape=(batch_size, nb_classes))
loss = -(logit_y - tf.tile(y_target, [K, 1]))**2 - tf.reduce_mean(true_logit)
g = tf.gradients(loss, x)[0]

eps = 8.0/255.0
N = 200
step_size = 4 * eps / N

X_test_adv = X_test[:num_batches*batch_size].copy()
Y_test_adv = Y_test.copy()

clean_logits = sess.run(logit_y, feed_dict={x: X_test[:batch_size], y: Y_test[:batch_size]})
assert np.argmax(clean_logits[0]) != 8
assert np.argmax(clean_logits[1]) == 8

for j in range(num_batches):
    X = X_test[j*batch_size: (j+1)*batch_size]
    Y = Y_test[j*batch_size: (j+1)*batch_size]

    # pick some target logits from clean examples of a different class
    Y_target = np.zeros((len(X), nb_classes), dtype=np.float32)
    for i in range(len(X)):
        c = np.argmax(Y[i])
        if c == 8:
            Y_target[i] = clean_logits[0]
        else:
            Y_target[i] = clean_logits[1]
    
    X_adv = X + np.random.uniform(low=-eps, high=eps, size=X.shape)
    X_adv = np.clip(X_adv, 0, 1)

    best_adv = X.copy()
    success = np.zeros(len(X), dtype=np.bool)

    for i in range(N):
        loss_np, g_np, preds_np = sess.run([loss, g, preds], feed_dict={x: X_adv, y: Y, y_target: Y_target})

        # check if examples are detected
        y_adv = np.zeros((preds_np.shape[0], nb_classes), dtype=np.float32)
        y_adv[np.arange(preds_np.shape[0]), np.argmax(preds_np, 1)] = 1 
        tps = kl_test(y_adv, preds_np, np.ones(len(Y), dtype=np.bool), train_stats)
        misclass = np.argmax(preds_np, axis=-1) != np.argmax(Y, axis=-1)

        # retain examples that fool the classifier and KL-detector
        best_adv[misclass & ~tps] = X_adv[misclass & ~tps]
        success[misclass & ~tps] = True

        if i % 20 == 0:
            print(i, np.mean(loss_np), np.mean(misclass), np.mean(misclass & ~tps), np.mean(success))
        
        X_adv = X_adv + step_size * np.sign(g_np)
        X_adv = np.clip(X_adv, X-eps, X+eps)
        X_adv = np.clip(X_adv, 0, 1)
    
    X_test_adv[j*batch_size: (j+1)*batch_size] = best_adv.copy()
    Y_test_adv[j*batch_size: (j+1)*batch_size] = sess.run(preds, feed_dict={x: best_adv})

succ = np.mean(np.argmax(Y_test_adv, axis=-1) != np.argmax(Y_test, axis=-1))
print("Attack success on classifier: {:.1f}%".format(100.0 * succ))

# 
# Check how many of our adversarial examples on the classifier are detected (at a 5% FP rate)
#
y_logits_adv = Y_test_adv
y_adv = np.zeros((y_logits_adv.shape[0], nb_classes), dtype=np.float32)
y_adv[np.arange(y_logits_adv.shape[0]), np.argmax(y_logits_adv, 1)] = 1 
tps = kl_test(y_adv, y_logits_adv, np.ones(len(y_logits_adv), dtype=np.bool), train_stats)
print("TP rate for adv examples: {:.1f}%".format(100 * np.mean(tps)))

0 -1780.0818 0.105 0.09 0.09
20 -565.1321 0.685 0.65 0.76
40 -204.31625 0.935 0.905 0.975
60 -74.7311 0.995 0.99 0.995
80 -28.34605 1.0 0.985 1.0
100 -9.395645 0.995 0.985 1.0
120 -1.0101721 0.995 0.985 1.0
140 3.2002714 0.995 0.985 1.0
160 5.6467557 0.995 0.99 1.0
180 7.2467613 1.0 0.995 1.0
0 -1726.4948 0.1 0.085 0.085
20 -555.5846 0.695 0.65 0.805
40 -201.71548 0.96 0.925 0.99
60 -79.84 0.98 0.96 0.995
80 -33.875984 0.985 0.98 0.995
100 -14.344636 0.995 0.985 1.0
120 -5.719374 0.995 0.99 1.0
140 -1.2585274 0.99 0.985 1.0
160 1.35889 0.995 0.985 1.0
180 3.1532295 1.0 0.995 1.0
0 -1884.9523 0.09 0.085 0.085
20 -591.1105 0.72 0.66 0.775
40 -222.87372 0.945 0.91 0.985
60 -88.07536 0.99 0.96 1.0
80 -33.886524 0.995 0.99 1.0
100 -11.766314 1.0 0.99 1.0
120 -1.8409826 1.0 1.0 1.0
140 3.46654 1.0 1.0 1.0
160 6.5058165 1.0 1.0 1.0
180 8.365455 1.0 0.985 1.0
0 -1798.8575 0.15 0.14 0.14
20 -574.13513 0.72 0.68 0.78
40 -224.9098 0.925 0.885 0.975
60 -96.50799 0.985 0.97 0.995
80 -44.00956 0.995

# A simpler attack: feature adversaries

As the generative classifier is built on top of a feature extractor,
we'll try to just get the feature extractor to match the features of a different class

In [23]:
# first build the feature extractor
from defense.vgg_cifar10 import cifar10vgg
cnn = cifar10vgg('models/cifar10vgg.h5', train=False)
N_layer = 36

def feature_extractor(x):
    out = cnn.normalize_production(x * 255.0)
    for i in range(N_layer):
        out = cnn.model.layers[i](out)
    if len(out.get_shape().as_list()) == 4:
        out = tf.reshape(out, [x.get_shape().as_list()[0], -1])
    return out

# minimize the squared loss over features
fea = feature_extractor(x)
fea_target_ph = tf.placeholder(tf.float32, shape=fea.get_shape().as_list())
loss1 = -(fea_target_ph - fea)**2
g1 = tf.gradients(loss1, x)[0]

eps = 8.0/255.0
N = 200
step_size = 4 * eps / N

X_test_adv = X_test[:num_batches*batch_size].copy()
Y_test_adv = Y_test.copy()

# get features for clean examples
clean_fea = sess.run(fea, feed_dict={x: X_test[:batch_size], y: Y_test[:batch_size]})

for j in range(num_batches):
    X = X_test[j*batch_size: (j+1)*batch_size]
    Y = Y_test[j*batch_size: (j+1)*batch_size]

    # pick some target features from clean examples of a different class
    fea_target = np.zeros((len(X), clean_fea.shape[1]), dtype=np.float32)
    for i in range(len(X)):
        c = np.argmax(Y[i])
        if c == 8:
            fea_target[i] = clean_fea[0]
        else:
            fea_target[i] = clean_fea[1]
    
    X_adv = X + np.random.uniform(low=-eps, high=eps, size=X.shape)
    X_adv = np.clip(X_adv, 0, 1)

    best_adv = X.copy()
    success = np.zeros(len(X), dtype=np.bool)

    for i in range(N):
        loss_np, g_np, preds_np = sess.run([loss1, g1, preds], feed_dict={x: X_adv, y: Y, fea_target_ph: fea_target})

        y_adv = np.zeros((preds_np.shape[0], nb_classes), dtype=np.float32)
        y_adv[np.arange(preds_np.shape[0]), np.argmax(preds_np, 1)] = 1 
        tps = kl_test(y_adv, preds_np, np.ones(len(Y), dtype=np.bool), train_stats)
        misclass = np.argmax(preds_np, axis=-1) != np.argmax(Y, axis=-1)

        best_adv[misclass & ~tps] = X_adv[misclass & ~tps]
        success[misclass & ~tps] = True

        if i % 20 == 0:
            print(i, np.mean(loss_np), np.mean(misclass), np.mean(misclass & ~tps), np.mean(success))
        X_adv = X_adv + step_size * np.sign(g_np)
        X_adv = np.clip(X_adv, X-eps, X+eps)
        X_adv = np.clip(X_adv, 0, 1)
    
    X_test_adv[j*batch_size: (j+1)*batch_size] = best_adv.copy()
    Y_test_adv[j*batch_size: (j+1)*batch_size] = sess.run(preds, feed_dict={x: best_adv})

succ = np.mean(np.argmax(Y_test_adv, axis=-1) != np.argmax(Y_test, axis=-1))
print("Attack success on classifier: {:.1f}%".format(100.0 * succ))

# 
# Check how many of our adversarial examples on the classifier are detected (at a 5% FP rate)
#
y_logits_adv = Y_test_adv
y_adv = np.zeros((y_logits_adv.shape[0], nb_classes), dtype=np.float32)
y_adv[np.arange(y_logits_adv.shape[0]), np.argmax(y_logits_adv, 1)] = 1 
tps = kl_test(y_adv, y_logits_adv, np.ones(len(y_logits_adv), dtype=np.bool), train_stats)
print("TP rate for adv examples: {:.1f}%".format(100 * np.mean(tps)))

0 -2.2959085 0.12 0.105 0.105
20 -1.2009311 0.49 0.455 0.71
40 -0.8690191 0.875 0.83 0.965
60 -0.68142265 0.96 0.955 0.995
80 -0.5690765 0.985 0.97 0.995
100 -0.4989743 0.995 0.98 1.0
120 -0.45456243 0.995 0.99 1.0
140 -0.42437187 0.995 0.99 1.0
160 -0.40333596 0.995 0.99 1.0
180 -0.38730893 0.995 0.995 1.0
0 -2.2622085 0.1 0.09 0.09
20 -1.1723541 0.575 0.525 0.715
40 -0.85416776 0.88 0.825 0.96
60 -0.67046523 0.955 0.935 0.985
80 -0.5590899 0.98 0.97 1.0
100 -0.48979488 0.985 0.985 1.0
120 -0.44529638 0.99 0.985 1.0
140 -0.4145382 0.99 0.99 1.0
160 -0.3926131 0.99 0.985 1.0
180 -0.37640268 0.995 0.995 1.0
0 -2.3835235 0.09 0.08 0.08
20 -1.1905704 0.63 0.54 0.76
40 -0.8687002 0.865 0.85 0.965
60 -0.6812602 0.985 0.965 1.0
80 -0.56381774 1.0 0.99 1.0
100 -0.48981556 1.0 0.995 1.0
120 -0.44198307 1.0 0.995 1.0
140 -0.40900207 1.0 1.0 1.0
160 -0.38566655 1.0 0.995 1.0
180 -0.36893216 1.0 1.0 1.0
0 -2.3291051 0.14 0.12 0.12
20 -1.1954292 0.59 0.505 0.755
40 -0.8698534 0.9 0.87 0.955
60 -0.